<h1 style="background-color:pink;font-family:newtimeroman;font-size:550%;text-align:center;border-radius: 15px 50px;padding: 5px">GENDER PREDICTION</h1>


<center><img src="https://miro.medium.com/max/875/1*8-1QrV3nlRjTFlDRJs2swg.jpeg",height='500',width='600'></center>


<h1 style="background-color:lightgreen;font-family:newtimeroman;font-size:400%;text-align:center;border-radius: 15px 50px;padding: 3px">OverView </h1>
<a id=0></a>

In this dataset our main objective is to predict the gender of the person by analysing his/her images. Here we have about 24k labeled examples, by using this we have to make a generalised model for predicting the gender of the person. The column pixels contains the value of pixels of the image in string format separated by a space. For gender column 0 represents 'Male' and 1 represents 'Female'.

<h1 style="background-color:lightgreen;font-family:newtimeroman;font-size:400%;text-align:center;border-radius: 15px 50px;padding: 3px">Importing modules and Loading datasets</h1>
<a id=1></a>

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import warnings
from sklearn.model_selection import train_test_split
import keras
from keras.layers import Dense,Conv2D,Flatten,MaxPool2D,BatchNormalization
from keras.callbacks import EarlyStopping,LearningRateScheduler,ReduceLROnPlateau
from keras.optimizers import SGD,RMSprop
from sklearn.metrics import classification_report,confusion_matrix
from keras.models import Sequential
warnings.filterwarnings('ignore')

In [ ]:
data=pd.read_csv('../input/age-gender-and-ethnicity-face-data-csv/age_gender.csv')
data.head()

In [ ]:
def value_to_image(pixels):
    pixels=np.array(pixels.split(),'float64')
    pixels=np.reshape(pixels,(48,48))
    pixels=pixels/255.0
    return pixels
data['pixels']=data['pixels'].apply(value_to_image)

<h1 style="background-color:lightgreen;font-family:newtimeroman;font-size:400%;text-align:center;border-radius: 15px 50px;padding: 3px">Train test spliting of data</h1>
<a id=2></a>

In [ ]:
train_img,test_img,train_gender,test_gender=train_test_split(data['pixels'],data['gender'],test_size=0.15,random_state=42)

<h1 style="background-color:lightgreen;font-family:newtimeroman;font-size:400%;text-align:center;border-radius: 15px 50px;padding: 3px">Visualising Data</h1>
<a id=3></a>

In [ ]:
gender_names=train_gender.replace({0:'Male',1:'Female'})

In [ ]:
plt.figure(figsize=(20,15))
for i in range(0,8):
    plt.subplot(2,4,(i%8)+1)
    num=np.random.randint(1000)
    plt.title("image no.{0} , gender:-{1}".format(i+1,gender_names.iloc[num]))
    plt.imshow(train_img.iloc[num],cmap='gray')
plt.show()

<h1 style="background-color:lightgreen;font-family:newtimeroman;font-size:400%;text-align:center;border-radius: 15px 50px;padding: 3px">Changing Dimension Of Data</h1>
<a id=4></a>

In [ ]:
def change_image_dimension(data):
    data=np.reshape(data.to_list(),(len(data),48,48,1))
    return data

In [ ]:
train_img=change_image_dimension(train_img)
test_img=change_image_dimension(test_img)
# train_img
train_img=train_img/255.0
test_img=test_img/255.0

<h1 style="background-color:lightgreen;font-family:newtimeroman;font-size:400%;text-align:center;border-radius: 15px 50px;padding: 3px">Callback Functions</h1>
<a id=5></a>

In [ ]:
early_stop=EarlyStopping(monitor='val_accuracy',patience=3)
reducelr=ReduceLROnPlateau(monitor='val_accuracy',patience=1,min_lr=0.000001)

<h1 style="background-color:lightgreen;font-family:newtimeroman;font-size:400%;text-align:center;border-radius: 15px 50px;padding: 3px">Convolutional Neural Network</h1>
<a id=5></a>

In [ ]:
model=Sequential()
model.add(Conv2D(256,(3,3),activation='relu',padding='same',input_shape=(48,48,1)))
model.add(MaxPool2D(2,2))
model.add(BatchNormalization())
model.add(Conv2D(64,(3,3),activation='relu',padding='same'))
model.add(MaxPool2D(2,2))
model.add(BatchNormalization())
model.add(Conv2D(32,(3,3),activation='relu',padding='same'))
model.add(MaxPool2D(2,2))
model.add(BatchNormalization())
model.add(Flatten())
model.add(Dense(512,activation='relu'))
model.add(Dense(64,activation='relu'))
model.add(Dense(1,activation='sigmoid'))

In [ ]:
model.compile(optimizer=SGD(0.01),loss='binary_crossentropy',metrics=['accuracy'])
r2=model.fit(train_img,train_gender,validation_data=(test_img,test_gender),epochs=20,callbacks=[reducelr,early_stop])

<h1 style="background-color:lightgreen;font-family:newtimeroman;font-size:400%;text-align:center;border-radius: 15px 50px;padding: 3px">Model Summary</h1>
<a id=7></a>

In [ ]:
model.summary()

<h1 style="background-color:lightgreen;font-family:newtimeroman;font-size:400%;text-align:center;border-radius: 15px 50px;padding: 3px">Plotting Losses</h1>
<a id=8></a>

In [ ]:
plt.plot(r2.history['loss'])
plt.plot(r2.history['val_loss'])

<h1 style="background-color:lightgreen;font-family:newtimeroman;font-size:400%;text-align:center;border-radius: 15px 50px;padding: 3px">Plotting Accuracy</h1>
<a id=9></a>

In [ ]:
plt.plot(r2.history['accuracy'])
plt.plot(r2.history['val_accuracy'])

<h1 style="background-color:lightgreen;font-family:newtimeroman;font-size:400%;text-align:center;border-radius: 15px 50px;padding: 3px">Confusion Matrix</h1>
<a id=10></a>

In [ ]:
pred_train_gender=model.predict_classes(train_img)
pred_test_gender=model.predict_classes(test_img)

In [ ]:
cm_train=confusion_matrix(train_gender,pred_train_gender)
cm_test=confusion_matrix(test_gender,pred_test_gender)

In [ ]:
sns.heatmap(cm_train,annot=True)
plt.show()

In [ ]:
sns.heatmap(cm_test,annot=True)
plt.show()

<h1 style="background-color:lightgreen;font-family:newtimeroman;font-size:400%;text-align:center;border-radius: 15px 50px;padding: 3px">Visualizing Mispredicted Training Examples</h1>
<a id=11></a>

In [ ]:
train_gender=np.array(train_gender,'int32')
pred_train_gender=np.reshape(pred_train_gender,(len(train_gender),))
train_gender=np.reshape(train_gender,(len(train_gender),))

In [ ]:
misclassified_train_index=np.where(train_gender!=pred_train_gender)
images=np.reshape(train_img,(len(train_img),48,48))

In [ ]:
plt.figure(figsize=(20,15))
for i in range(0,5):
    plt.subplot(1,4,(i%4)+1)
    pred='Male' if pred_train_gender[misclassified_train_index[0][i]]==0 else 'Female'
    actual='Male' if train_gender[misclassified_train_index[0][i]]==0 else 'Female'
    plt.title("classified as {0} but the person is {1}".format(pred,actual))
    plt.imshow(images[misclassified_train_index[0][i]],cmap='gray')
plt.show()

<h1 style="background-color:lightgreen;font-family:newtimeroman;font-size:400%;text-align:center;border-radius: 15px 50px;padding: 3px">Visualizing Mispredicted Test Examples</h1>
<a id=12></a>

In [ ]:
test_gender=np.array(test_gender,'int32')
pred_test_gender=np.reshape(pred_test_gender,(len(test_gender),))
test_gender=np.reshape(test_gender,(len(test_gender),))

In [ ]:
pred_test_gender=np.reshape(pred_test_gender,(len(test_gender),))
misclassified_test_index=np.where(test_gender!=pred_test_gender)
test_images=np.reshape(test_img,(len(test_img),48,48))

In [ ]:
plt.figure(figsize=(20,15))
for i in range(0,5):
    plt.subplot(1,4,(i%4)+1)
    pred='Male' if pred_test_gender[misclassified_test_index[0][i]]==0 else 'Female'
    actual='Male' if test_gender[misclassified_test_index[0][i]]==0 else 'Female'
    plt.title("classified as {0} but the person is {1}".format(pred,actual))
    plt.imshow(images[misclassified_test_index[0][i]],cmap='gray')
plt.show()

<h1 style="background-color:lightgreen;font-family:newtimeroman;font-size:400%;text-align:center;padding: 3px">CONCLUSION</h1>
<h3 style="background-color:lightblue;font-family:newtimeroman;font-size:200%;text-align:center;padding: 3px">We can see that the model predicts quite well with accuracy on test data as about 90% and train data accuracy of about 93%</h3>
<a id=13></a>

<h4 style="background-color:lightyellow;font-family:newtimeroman;font-size:200%;text-align:center;padding: 3px">If you liked the notebook.. please upvote and if have any query feel free to ask in comment section.</h4>
